In [1]:
pwd

'C:\\Users\\ADMIN\\Documents\\GitHub\\python-for-all\\Time-series'

In [28]:
cd E:\Time-Series Data

E:\Time-Series Data


In [13]:
# Importing libraties
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import time, timedelta
import seaborn as sns
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [29]:
#df_train = pd.read_csv("train.csv", usecols=[1, 2, 3, 4],
                       #converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
                       #parse_dates=["date"])

df_train = pd.read_csv("train.csv",usecols=[1, 2, 3, 4,5],dtype={'onpromotion': bool},
                       converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
                       parse_dates=["date"], skiprows=range(1, 66458909))  # 2016-01-01

In [30]:
df_train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,2.564949,False
1,2016-01-01,25,105575,2.302585,False
2,2016-01-01,25,105857,1.386294,False
3,2016-01-01,25,108634,1.386294,False
4,2016-01-01,25,108701,1.098612,True


In [31]:
df_train.shape

(59038132, 5)

In [32]:
df_test = pd.read_csv("test.csv",usecols=[0, 1, 2, 3, 4],dtype={'onpromotion': bool},
                      parse_dates=["date"]).set_index(['store_nbr', 'item_nbr', 'date'])

In [34]:
items = pd.read_csv("items.csv").set_index("item_nbr")

In [35]:
df_2017 = df_train[df_train.date.isin(pd.date_range("2017-05-31", periods=7 * 11))].copy()

In [36]:
del df_train

In [37]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
50912462,2017-05-31,1,96995,0.693147,False
50912463,2017-05-31,1,99197,0.693147,False
50912464,2017-05-31,1,103520,1.386294,False
50912465,2017-05-31,1,103665,2.197225,False
50912466,2017-05-31,1,105574,1.386294,False


In [38]:
promo_2017_train = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)

In [39]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

In [40]:
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)

In [41]:
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)

In [42]:
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

In [43]:
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

In [44]:
del promo_2017_test, promo_2017_train

In [45]:
df_2017 = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)

In [46]:
df_2017.columns = df_2017.columns.get_level_values(1)

In [47]:
items = items.reindex(df_2017.index.get_level_values(1))

In [48]:
def get_timespan(df, dt, minus, periods):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods)]

In [55]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values})
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[pd.date_range(t2017, periods=16)].values
        return X, y
    return X

In [56]:
t2017.head()

NameError: name 't2017' is not defined

In [50]:
print("Preparing dataset...")

Preparing dataset...


In [57]:
# t2017 = date(2017, 6, 21)

t2017 = date(2017,6,21)

NameError: name 'date' is not defined

In [52]:
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(t2017 + delta)
    X_l.append(X_tmp)
    y_l.append(y_tmp)

NameError: name 't2017' is not defined

In [ ]:
X_train = pd.concat(X_l, axis=0)

In [ ]:
y_train = np.concatenate(y_l, axis=0)

In [ ]:
del X_l, y_l

In [ ]:
X_val, y_val = prepare_dataset(date(2017, 7, 26))

In [ ]:
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

In [ ]:
print("Training and predicting models...")

In [ ]:
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}
MAX_ROUNDS = 1000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 4) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

In [ ]:
print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

In [ ]:
print("Making submission...")

In [ ]:
y_test = np.array(test_pred).transpose()

In [ ]:
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")

In [ ]:
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [ ]:
submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)
